# Building and running

In [1]:
import pandas as pd
import numpy as np
import cv2
import hashlib

from tensorflow import keras

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

## Load and prepare dataset
The dataset is composed by:
 * CSV with the labeling
 * Image folder with all the images normalized

In [2]:
dataset_folder = 'normalized_data_set_diagrams/'
labeled_csv = 'csv/diagram_images_dataset.csv'

In [3]:
map_to_name = {}


def load_dataset(dataset_folder_path, csv_path):
    """Loads a dataset of images
        - dataset_folder_path is the path of the folder that contains the images
        - csv_path is the path of the CSV file that contains the labels of the images
        Returns: X_data, y_labeled
        - X_data is a numpy.ndarray containing the pixel data of an image X
        - y_labeled is a numpy.ndarray containing an int, the label Y for the image X in that index
    """
    X_data = []

    data = pd.read_csv(csv_path, dtype={"Name": str, "Category": np.uint8})

    for image_name in data.Name:
        img = cv2.imread(dataset_folder_path + image_name, cv2.IMREAD_COLOR)
        hash = hashlib.sha1(img).hexdigest()[:15]
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        map_to_name[hash] = image_name
        X_data.append(img)

    X_data = np.array(X_data)
    y_labeled = np.array(data.Category)

    print("Data loaded\n", data)
    return X_data, y_labeled

In [4]:
def preprocess_data(x, y, m):
    """Pre-processes the data for the model
        - x is a numpy.ndarray of shape (m, 224, 224, 3) containing
         a list of image pixels, where m is the number of images
        - y is a numpy.ndarray of shape (m,) containing
         the labels for x
        - m is the number of categories in the classifier
        Returns: X_p, Y_p
        - X_p is a numpy.ndarray containing the preprocessed X
        - Y_p is a numpy.ndarray containing the preprocessed Y
    """
    X_p = keras.applications.densenet.preprocess_input(x)

    y_p = keras.utils.to_categorical(y, m)

    return X_p, y_p

In [5]:
X_full, y_full = load_dataset(dataset_folder, labeled_csv)
X_full_p, y_full_p = preprocess_data(X_full, y_full, 7)

Data loaded
                      Name  Category
0     7a668879d103ba8.jpg         1
1     4bab7d342c24e3f.jpg         1
2     d6c5e6d46cbbb26.jpg         1
3     e215c30192cc297.jpg         1
4     0fd2b9ef096d9cb.jpg         1
...                   ...       ...
5551  d2254621efd8d52.jpg         0
5552  de2268621d5e911.jpg         0
5553  90fe34f3f8107ee.jpg         0
5554  f065035fafcf430.jpg         0
5555  30efd0c8a1649e1.jpg         0

[5556 rows x 2 columns]


## Building DenseNet169

In [6]:
input_shape_densenet = (224, 224, 3)

In [10]:
def build_network(trainable: bool, retrain_last: bool):
    """Pre-processes the data for the model
        - trainable boolean to indicate if the network would be fully trainable
        - retrain_last boolean to indicate if the last layer would be trainable
        Returns: densenet_model
        - densenet_model is a Keras DenseNet169 model
    """
    densenet_model = keras.applications.DenseNet169(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=input_shape_densenet,
        pooling=None
    )

    densenet_model.trainable = True

    if not trainable:
        for layer in densenet_model.layers:
            if retrain_last and 'conv5' in layer.name:
                layer.trainable = True
            else:
                layer.trainable = False

    return densenet_model

### Add new layers

In [11]:
def add_extra_layers(densenet_model, layer_size: int, dropout: bool, number_of_layers: int):
    """Add extra layers to a Keras model for transfer learning.
        - densenet_model is a pre-trained Keras model with input (224, 224, 3)
        - layer_size is an int, the size of the first Dense layer
        - dropout is a bool, indicating if a Dropout layer will be added
          between Dense layers
        Returns: model
        - model a Keras model with the layer added
    """
    initializer = keras.initializers.he_normal(seed=32)
    inputs = keras.Input(shape=input_shape_densenet)

    layer = densenet_model(inputs)
    layer = keras.layers.Flatten()(layer)

    layer = keras.layers.BatchNormalization()(layer)

    for n in range(1, number_of_layers + 1):
        layer = keras.layers.Dense(units=layer_size / n,
                                   activation='relu',
                                   kernel_initializer=initializer
                                   )(layer)
        if dropout:
            layer = keras.layers.Dropout(0.5)(layer)

        layer = keras.layers.BatchNormalization()(layer)

    layer = keras.layers.Dense(units=7,
                               activation='softmax',
                               kernel_initializer=initializer
                               )(layer)

    model = keras.models.Model(inputs, outputs=layer)
    model.summary()

    return model

## Validate results

In [15]:
def train_with_k_fold_cross_validation(n_folds: int, layer_size: int, trainable: bool, retrain_last: bool,
                                       dropout: bool, number_of_layers: int):
    """Create and train a DenseNet model n_folds times with a different training/validation partition data.
        - n_folds the number of times the model will be trained
        - layer_size is an int, the size of the first Dense layer
        - trainable boolean to indicate if the network would be fully trainable
        - retrain_last boolean to indicate if the last layer would be trainable
        - dropout is a bool, indicating if a Dropout layer will be added
          between Dense layers
        Returns: histories
        - histories a list of size n_folds with the detailed training history of each attempt
    """
    histories = []
    for fold in range(n_folds):
        network = build_network(trainable, retrain_last)
        network = add_extra_layers(network, layer_size, dropout, number_of_layers)
        network.compile(loss='categorical_crossentropy',
                        optimizer=keras.optimizers.Adam(),
                        metrics=['accuracy'])

        X_train, X_val, y_train, y_val = train_test_split(X_full_p, y_full_p, test_size=0.2, random_state=fold * 5)
        history = network.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=2)
        del network
        histories.append(history)

    return histories

In [12]:
def train_with_simple_holdout_validation(X_train, X_val, y_train, y_val):
    network = build_network(True)
    network = add_extra_layers(network, 128, True, 1)
    network.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adam(),
                    metrics=['accuracy'])

    network.fit(X_train, y_train, epochs=9, validation_data=(X_val, y_val), verbose=1)

    network.save('diagrams.h5')
    return network

### Refinement

In [ ]:
folds = 4

Helper functions to plot the results

In [ ]:
def get_history_mean(k_history, prop: str):
    mean_prop = [h.history[prop] for h in k_history]
    mean = np.mean(mean_prop, axis=0)
    return mean

In [11]:
def plot_model_accuracy(accs, losses, legends):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    for acc_value in accs:
        plt.plot(acc_value)
        l = len(acc_value) - 1
        plt.text(l, acc_value[l], "{:.1f}%".format(acc_value[l] * 100))
    plt.legend(legends, loc='lower right')
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')

    plt.subplot(1, 2, 2)
    for loss_value in losses:
        plt.plot(loss_value)
        l = len(loss_value) - 1
        plt.text(l, loss_value[l], "{:.1f}%".format(loss_value[l] * 100))
    plt.legend(legends, loc='upper right')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.savefig('number_of_layers.png')
    plt.show()

In [ ]:
def plot_model_comparison(history, legends):
    val_acc = map(get_history_mean, history, ['val_accuracy'] * len(history))
    val_loss = map(get_history_mean, history, ['val_loss'] * len(history))

    plot_model_accuracy(val_acc, val_loss, legends)

#### Number of extra layers

In [ ]:
def extra_layers_test():
    history = []
    first_layer_size = 128
    fully_trainable = False
    retrain_last = True
    dropout = True
    n_layers = [0, 1, 2]
    for n_layer in n_layers:
        h = np.array(train_with_k_fold_cross_validation(folds, first_layer_size, fully_trainable, retrain_last, dropout, n_layer))
        history.append(h)
    plot_model_comparison(history, n_layers)

In [ ]:
extra_layers_test()

#### Size of the first layer

In [ ]:
def first_layer_size_test():
    history = []
    first_layer_size = [512, 256, 128, 64]
    fully_trainable = False
    retrain_last = True
    dropout = True
    n_layers = 1
    for layer_size in first_layer_size:
        h = np.array(train_with_k_fold_cross_validation(folds, layer_size, fully_trainable, retrain_last, dropout, n_layers))
        history.append(h)
    plot_model_comparison(history, first_layer_size)

In [ ]:
first_layer_size_test()

#### Dropout

In [ ]:
def dropout_test():
    history = []
    first_layer_size = 128
    fully_trainable = False
    retrain_last = True
    dropout = [True, False]
    n_layers = 1
    for option in dropout:
        h = np.array(train_with_k_fold_cross_validation(folds, first_layer_size, fully_trainable, retrain_last, option, n_layers))
        history.append(h)
    plot_model_comparison(history, ['Dropout', 'Without'])

In [ ]:
dropout_test()

#### Retrain last layer

In [ ]:
def retrain_last_layer_test():
    history = []
    first_layer_size = 128
    fully_trainable = False
    retrain_last = [True, False]
    dropout = True
    n_layers = 1
    for option in retrain_last:
        h = np.array(train_with_k_fold_cross_validation(folds, first_layer_size, fully_trainable, option, dropout, n_layers))
        history.append(h)
    plot_model_comparison(history, ['Retrained', 'Frozen'])

In [ ]:
retrain_last_layer_test()

### Fine-tuning

In [ ]:
def generate_final_model(seed):
    X_train, X_val, y_train, y_val = train_test_split(X_full_p, y_full_p, test_size=0.2, random_state=seed)
    _, X_original, _, _ = train_test_split(X_full, y_full, test_size=0.2, random_state=seed)

    model = train_with_simple_holdout_validation(X_train, X_val, y_train, y_val)
    return model

In [18]:
loaded_model = keras.models.load_model('diagrams.h5')
loaded_model.trainable = True
loaded_model.compile(loss='categorical_crossentropy',
                     optimizer=keras.optimizers.Adam(1e-5),
                     metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(X_full_p, y_full_p, test_size=0.2, random_state=20)
loaded_model.summary()
history = loaded_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=1)
history

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 batch_normalization (BatchN  (None, 81536)            326144    
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 128)               10436736  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                             

2022-09-01 01:28:33.298427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


139/139 [==============================] - ETA: 0s - loss: 0.1081 - accuracy: 0.9653

2022-09-01 01:30:50.807744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


139/139 [==============================] - 165s 1s/step - loss: 0.1081 - accuracy: 0.9653 - val_loss: 0.0196 - val_accuracy: 0.9946
Epoch 2/10
139/139 [==============================] - 133s 946ms/step - loss: 0.0407 - accuracy: 0.9863 - val_loss: 0.0185 - val_accuracy: 0.9910
Epoch 3/10
139/139 [==============================] - 133s 951ms/step - loss: 0.0303 - accuracy: 0.9901 - val_loss: 0.0153 - val_accuracy: 0.9928
Epoch 4/10
139/139 [==============================] - 134s 960ms/step - loss: 0.0243 - accuracy: 0.9935 - val_loss: 0.0157 - val_accuracy: 0.9928
Epoch 5/10
139/139 [==============================] - 133s 955ms/step - loss: 0.0187 - accuracy: 0.9937 - val_loss: 0.0148 - val_accuracy: 0.9937
Epoch 6/10
139/139 [==============================] - 133s 954ms/step - loss: 0.0108 - accuracy: 0.9980 - val_loss: 0.0148 - val_accuracy: 0.9937
Epoch 7/10
139/139 [==============================] - 134s 960ms/step - loss: 0.0116 - accuracy: 0.9966 - val_loss: 0.0138 - val_accuracy:

### Validating failed cases and measure model performance

In [21]:
def validate_failed(model, X, y, X_original, show_image: bool):
    prob = model.predict(X, verbose=1)
    predictions = prob.argmax(axis=-1)
    expected_y = y.argmax(axis=-1)
    fails = 0
    for i in range(len(predictions)):
        if predictions[i] != expected_y[i]:
            fails += 1
            if show_image:
                hash = hashlib.sha1(X_original[i]).hexdigest()[:15]
                name = map_to_name[hash]
                print(
                    f'\r{name} Expected {expected_y[i]} ({prob[i][expected_y[i]]}) but got {predictions[i]} ({prob[i][predictions[i]]})',
                    flush=True, end=' ' * 50)
                cv2.imshow('Failed', X_val[i])
                cv2.imshow('Original', X_original[i])
                cv2.waitKey(0)
    print(f'Failed: {fails}')

    return classification_report(expected_y, predictions, digits=4)

In [22]:
failed = validate_failed(loaded_model, X_val, y_val, X_original, False)
print(failed)

2022-09-01 01:52:22.184183: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 17s 399ms/step
Failed: 0
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       205
           1     1.0000    1.0000    1.0000       115
           2     1.0000    1.0000    1.0000       160
           3     1.0000    1.0000    1.0000       208
           4     1.0000    1.0000    1.0000        68
           5     1.0000    1.0000    1.0000       169
           6     1.0000    1.0000    1.0000       187

    accuracy                         1.0000      1112
   macro avg     1.0000    1.0000    1.0000      1112
weighted avg     1.0000    1.0000    1.0000      1112



### Testing new cases

In [ ]:
loaded_model = keras.models.load_model('diagrams.h5')

In [27]:
def show_and_predict_img(name: str, model):
    img = cv2.imread('test/' + name, cv2.IMREAD_ANYCOLOR)
    im = np.array([img])
    im = keras.applications.densenet.preprocess_input(im)

    cv2.imshow('img', img)
    cv2.waitKey(0)

    prop = model.predict(im)
    prediction = prop.argmax(axis=-1)
    print(f"Prediction: {prediction}")

    return prop

In [57]:
loaded_model.save('tuned.h5')

In [56]:
show_and_predict_img('e78f463f0008003.jpg', loaded_model)

Prediction: [0]


array([[9.87176120e-01, 4.80586867e-04, 2.15088457e-05, 1.15415314e-02,
        5.98010025e-04, 7.62242125e-05, 1.05956795e-04]], dtype=float32)